#  Todo
* ~How do we keep an option to hover on the countries that have no data?~
* ~How do we hide the countries from the map that are *never* a part of the datasets? (Russia, Iceland etc)~
* How do we make a slider work?
* Should we do a "dias show"? In that case: How?
* Which datasets should we work with and which points do we want to make?

In [1]:
import pandas as pd

df = pd.read_csv("drug_offences.csv", sep=";")
df.head(3)

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,Unnamed: 13
0,Austria,47880.0,42610.0,36235.0,32907.0,30250.0,28227.0,23797.0,25892.0,23853.0,22729.0,20043.0,24166.0,NaN
1,Belgium,54749.0,51774.0,49416.0,47083.0,48727.0,42935.0,39181.0,41661.0,39307.0,41525.0,40357.0,37867.0,NaN
2,Bulgaria,2382.0,2433.0,4886.0,4195.0,9340.0,9521.0,8322.0,6167.0,6577.0,5149.0,4185.0,4301.0,NaN


In [21]:
df = df.set_index("Country")

In [82]:
new_df = []

for col, data in df.items():
    new_df.append([data.index.to_list(), data.values.tolist(), [data.name]*len(df)])

liist = [list(zip(x,y,z)) for x, y, z in new_df]
newlist = [item for items in liist for item in items]

In [102]:
large_df = pd.DataFrame(newlist, columns=["Country", "Value", "Year"])
large_df.head(2)

,Country,Value,Year
0,Austria,47880.0,2018
1,Belgium,54749.0,2018


In [121]:
fig = px.choropleth(large_df.loc[large_df.Year == "2016"], locations="Country",
                    color="Value", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    scope="europe",
                    locationmode="country names",
                    custom_data=["Country", "Value"])

fig.update_traces(hovertemplate="<b>Country: %{customdata[0]}<br></b><br>" +
                                     "Drug Offenders: %{customdata[1]:n}<br>", selector=dict(type='choropleth'))

empty_df = large_df.loc[large_df.Year == "2016"].query('Value != Value')

fig.add_trace(
go.Choropleth(locations=empty_df["Country"],
                    z=empty_df["Year"], # lifeExp is a column of gapminder
                    colorscale="Greys",
                    text=empty_df['Country'], # hover text
                    locationmode="country names",
                    hovertemplate="<b>Country: %{customdata}<br></b>No data available<extra></extra>",
                    customdata=[*empty_df["Country"]],
                    showlegend=None, showscale=False)
)

fig.update_geos(fitbounds="locations", visible=False)

In [137]:
def create_choropleth(df, year, colors="Plasma"):
    df = df.loc[df.Year == year]
    return go.Choropleth(locations=df["Country"],
                        z=df["Value"], # lifeExp is a column of gapminder
                        colorscale=colors,
                        locationmode="country names",
                        customdata=[*zip(df["Country"], df["Value"])])

def create_missing_choropleth(df, year, colors="Greys"):
    empty_df = large_df.loc[large_df.Year == "2016"].query('Value != Value')
    empty_df = empty_df.append({"Year": year, "Country":"Switzerland", "Value": 0}, ignore_index=True)
    empty_df[str(year)] = 0
    return go.Choropleth(locations=empty_df["Country"],
                        z=empty_df["Year"], # lifeExp is a column of gapminder
                        colorscale=colors,
                        locationmode="country names",
                        hovertemplate="<b>Country: %{customdata}<br></b>No data available<extra></extra>",
                        customdata=[*empty_df["Country"]],
                        showlegend=None, showscale=False)

year = "2016"
fig3 = go.Figure(create_choropleth(large_df, year))
fig3.update_traces(hovertemplate="<b>Country: %{customdata[0]}<br></b><br>" +
                                     "Drug Offenders: %{customdata[1]:n}<br><extra></extra>", selector=dict(type='choropleth'))
fig3.update_geos(scope="europe")
fig3.update_layout(title_text="Number of drug offences by country", width=500, height=400)

# Add missing
fig3.add_trace(create_missing_choropleth(large_df, year, "Greys"))
fig3.update_geos(fitbounds="locations", visible=False)
fig3.show()

In [12]:
import pandas as pd
import plotly
import numpy as np

data = [dict(type='choropleth',
             locations = df['Country'].astype(str),
             z=df[str(year)],
             locationmode='country names')]

# let's create the steps for the slider
steps = []
for i in range(len(data[0])):
    step = dict(method='restyle',
                args=['visible', [False] * len(data[0])],
                label='Year {}'.format(i + 1980))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]    
layout = dict(geo=dict(scope='europe',
                       projection={'type': 'mercator'}),
              sliders=sliders)

fig = dict(data=data, 
           layout=layout)
plotly.offline.iplot(fig)